In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
import pandas as pd
from ast import literal_eval

In [4]:
#Reading the CSV files that were previously generated for this purpose
df_train = pd.read_csv('Pre-processed_dataForAutoModelTokenClassification-trainData_new.csv',converters={'tokens':literal_eval, 'ner_tags':literal_eval})
df_validation = pd.read_csv('Pre-processed_dataForAutoModelTokenClassification-ValidationData_new.csv',converters={'tokens':literal_eval, 'ner_tags':literal_eval})

In [5]:
df_train

,id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans,tokens,label,ner_tags
0,0,die pandemie mag zu ende sein. viele geimpfte ...,negative,die pandemie,['viele geimpfte stecken weiter in ihren impfs...,...,...,"[die, pandemie, mag, zu, ende, sein, viele, ge...",0,"[0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1,"Wir sind hier nicht bei 'Wünsch dir was', sond...",negative,"['Klimaschutz', 'Habeck ']","['schlechtesten Bundesminister', 'Keinen Bock ...",...,...,"[Wir, sind, hier, nicht, bei, Wünsch, dir, was...",0,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, ..."
2,2,Da ist der falsche bestraft wurden,negative,...,der falsche bestraft wurden,...,...,"[Da, ist, der, falsche, bestraft, wurden]",0,"[2, 2, 0, 0, 0, 0]"
3,3,"Vor 500 Jahren hätte man gesagt, die #AfD sei ...",negative,#AfD,"['Man hätte sie geteert und gefedert, auf eine...",...,...,"[Vor, Jahren, hätte, man, gesagt, die, AfD, se...",0,"[2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, ..."
4,4,Die Ukrainer halten gerade ihren Kopf für uns ...,positive,...,...,...,Ukrainer,"[Die, Ukrainer, halten, gerade, ihren, Kopf, f...",1,"[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...,...,...,...
986,986,Zwei wahre Sätze für den Abend:\nJedes einzeln...,negative,Hamas,zivile Opfer im Gazastreifen,...,...,"[Zwei, wahre, Sätze, für, den, Abend, Jedes, e...",0,"[2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, ..."
987,987,Danke Südafrika👏 wenigstens schaut ihr nicht w...,positive,...,...,"['Danke Südafrika👏', 'seid keine Marionette']",...,"[Danke, Südafrika, 👏, wenigstens, schaut, ihr,...",1,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2]"
988,988,"Ohne angeben zu wollen, habe ich #Putin schon ...",negative,#Putin,"['Der Typ war schon immer total skrupellos', '...",...,...,"[Ohne, angeben, zu, wollen, habe, ich, Putin, ...",0,"[2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, ..."
989,989,"Bitte Deutschland noch mehr Waffen liefern, bi...",negative,...,"['getötet ', 'Aber ihr solltet auch nicht verg...",...,...,"[Bitte, Deutschland, noch, mehr, Waffen, liefe...",0,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, ..."


In [6]:
from datasets import load_dataset, DatasetDict,Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    'validation': Dataset.from_pandas(df_validation),
    "test": Dataset.from_pandas(df_test)
    })

/ltstorage/home/1samreen/miniconda3/envs/saint_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'sentiment', 'negative object_spans', 'negative spans', 'positive spans', 'positive object_spans', 'tokens', 'label', 'ner_tags'],
        num_rows: 991
    })
    validation: Dataset({
        features: ['id', 'text', 'sentiment', 'negative object_spans', 'negative spans', 'positive spans', 'positive object_spans', 'tokens', 'label', 'ner_tags'],
        num_rows: 141
    })
    test: Dataset({
        features: ['id', 'text', 'sentiment', 'negative object_spans', 'negative spans', 'positive spans', 'positive object_spans', 'tokens', 'label', 'ner_tags'],
        num_rows: 283
    })
})

### Token Classification - Fine tuning the model . Please refer to 'https://huggingface.co/docs/transformers/en/tasks/token_classification' for further information

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-xlm-roberta-base-sentiment')

/ltstorage/home/1samreen/miniconda3/envs/saint_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True,remove_columns=dataset['train'].column_names)

Map: 100%|██████████| 283/283 [00:00<00:00, 7995.66 examples/s]


In [11]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 991
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 141
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 283
    })
})

In [12]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [13]:
import evaluate

seqeval = evaluate.load("seqeval")

In [13]:
label_list = ['negative','positive','O']
label_list


['negative', 'positive', 'O']

In [15]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "accuracy": results["overall_accuracy"],
    }

In [14]:
id2label = {0: "negative",1: "positive", 2:"O"}
label2id = {"negative":0, "positive":1 , "O":2}

In [15]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "cardiffnlp/twitter-xlm-roberta-base-sentiment", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="fineTuningXLMRoberta-TokenClassification-Spacy",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_steps=50,
    save_steps= 50,
    eval_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
    #compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.731874
2,0.797600,0.685727
3,0.797600,0.686352
4,0.669900,0.741035
5,0.581200,0.716866
6,0.581200,0.736067
7,0.517200,0.754967
8,0.517200,0.769864
9,0.470500,0.790813
10,0.429200,0.798384


TrainOutput(global_step=465, training_loss=0.5075213719439763, metrics={'train_runtime': 291.5003, 'train_samples_per_second': 50.995, 'train_steps_per_second': 1.595, 'total_flos': 837957185025534.0, 'train_loss': 0.5075213719439763, 'epoch': 15.0})

In [ ]:
#Pushing the modelt o hugging face hub for use in future
trainer.push_to_hub()

In [ ]:
#Pushing the tokenizer to the same repository as the model
tokenizer.push_to_hub('<huggingfaceuser>/fineTuningXLMRoberta-TokenClassification-Spacy')

### Creating data using the above fine tuned model for NER Evaluation

In [21]:
import torch
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("Hina541/fineTuningXLMRoberta-TokenClassification-Spacy")
tokenizer = AutoTokenizer.from_pretrained("Hina541/fineTuningXLMRoberta-TokenClassification-Spacy")


In [43]:
from ast import literal_eval
df_test = pd.read_csv('Pre-processed_dataForAutoModelTokenClassification-TestData_new.csv',converters={'tokens':literal_eval, 'ner_tags':literal_eval, 'ner_tags_for_NEREval':literal_eval})

In [33]:
#Extracting label for each word using the fine tuned model
def Extract_label_for_each_word(text):
    tokens=[]
    encoded = tokenizer(text,padding=True,truncation=True,return_tensors="pt")
    word_ids = encoded.word_ids()
    with torch.no_grad():
        logits =model(**encoded).logits
    input_ids = encoded["input_ids"].tolist()[0]
    labels_appended =[]  
    for word_id in sorted(set(word_ids) - {None}):
        token_ids = [input_ids[i] for i in range(len(input_ids)) if word_ids[i]==word_id]
        word = tokenizer.decode(token_ids,clean_up_tokenization_spaces=True).strip()
        labels = [model.config.id2label[torch.argmax(logits[0][i]).item()] for i in range(len(input_ids)) if word_ids[i]==word_id]
        tokens.append(word)
        labels_appended.append(labels[0])
    return tokens,labels_appended
        

In [34]:
df_test[['words','labels_for_words']]=df_test["text"].apply(lambda x : pd.Series(Extract_label_for_each_word(x)))

In [77]:
#checking if the words are same as original spans and adding it to a new dataframe as 'tokens', 'actual sentiment' and 'predicted sentiment' for NER evaluation
tokens = []
actual_sentiment = []
predicted_sentiment =[]
emojis = ['🙁','😕','🇺🇦🤡🇩🇪🤣🤣🤣🤣🏆','💙💙💙🇩🇪🇩🇪🇩🇪🇩🇪💙💙💙','😡😡😡','🤓','😃','💙💙💙💙','😆','😎','😉😉','🙂','🤡','✋','🍀❤️','🙈','😁','🤝','🤡🤡','🕊','⬆️','😂😂😂😂😂','🤣🤦🏼','💔','👌🏻','🇵🇸','🔥','😤','🙏👍👏👏👏👏','🤷🏻','⛔️','😔','🇮🇱💪','😅🙃','🇮🇱🇮🇱🇮🇱','😪😪😪😪🍉🍉🍉','🇩🇪🇨🇭🇦🇹🇹🇷','🎉','💁','🤡🤦','😂🤣😅','🤪','👏👏','💸','😅','🤣🤣.','✌🏽','😞','!!!👍👍👍👍','🤝🏼','🤡🤡🤡🤡🤡🤡🤡🤡','🇩🇪🤝🏻🇺🇦','😂😂😂😂😂','🇩🇪💙🇺🇦💛🇪🇺','😳','👏','🌳🌳🌳','👍👍💙💙💙💙💪🇩🇪','💙🇮🇱','😜','🤣','😊☝️','🇦🇹❤️🇮🇱','🙄🙄','😬','🙄','🌱','😢😢','👍💛','😂😂','🇩🇪🤡','🤷','💪','👍','😱','🤦🏻','🤦🏼','🇵🇸','😕','😏','😥🙏','😂','😂.','😊','😠','😮','💫','✊🏼','👍🏼','🤔','😃','🌎🍀','😉','🌻','🇺🇦','😡😡','✌🏼','🕊️','😑','😆😆😆','🤨','🇩🇪','🤦','♂️🤦','😢','❌','💪💪💪','🇺🇸','🇮🇷','🏳️','🌈', '✌️🇵🇸♥️🇧🇷♥️🇿🇦✌️', '♂️🤡','🇵🇸💔💔💔','🛑','🚫!!!','👏😂','💚','🙂.','❌❌💔','🙈🙈🙈','🤦','🇩🇪🇮🇱🙏','🙏','🙇🏼','🌍♻️🔥💸💥🤷','👍!!','😂🤔😷','💚🔥','💚🌳','😔😔','😡','🥰','😂😂😂','🤡🤡🤡','🇮🇱','🙁🙁😔😔','🙌','😣','🤩','👍👍👍👍']
df_tokens_results = pd.DataFrame(columns=['tokens','actual_sentiment','predicted_sentiment'])
for i in range(0,len(df_test)):
    for index,token in enumerate(df_test['words'][i]):
        if token not in emojis:
            item = ''.join(e for e in token if e.isalnum())
        else:
            item = token
        if item != '':
            if item in df_test['negative object_spans'][i] or item in df_test['negative spans'][i] or item in df_test['positive object_spans'][i] or item in df_test['positive spans'][i] or item in emojis:
                #if item.strip().isalpha() or item.strip() in emojis:
                tokens.append(item)
                actual_sentiment.append(df_test['sentiment'][i])
                predicted_sentiment.append(df_test['labels_for_words'][i][index])
            else:
                tokens.append(item)
                actual_sentiment.append('O')
                predicted_sentiment.append('O')
    tokens.append(' ')
    actual_sentiment.append(' ')
    predicted_sentiment.append(' ')
df_tokens_results['tokens'] = tokens
df_tokens_results['actual_sentiment'] = actual_sentiment
df_tokens_results['predicted_sentiment'] = predicted_sentiment

In [79]:
#Saving the results to a csv for further ner evaluation
df_tokens_results.to_csv('ProperTokenClassification_Spacy2ndattempt.csv', sep='\t', index=False, header=False)

### Below are the results of NER Evaluation
#### python conlleval_perl.py -r < ProperTokenClassification_Spacy2ndattempt.csv -- command used for NER evaluation
#### processed 7556 tokens with 2827 phrases; found: 1687 phrases; correct: 1594.
#### accuracy:  83.68%; precision:  94.49%; recall:  56.38%; FB1:  70.62
         #### negative: precision:  95.37%; recall:  59.15%; FB1:  73.02  1511
         #### positive: precision:  86.93%; recall:  39.13%; FB1:  53.97  176